In [1]:
from qutip import*
from qutip.qip.operations import rx, ry, rz
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
import random

In [49]:
# define operator 

N = 40

Ic = qeye(N)
Iq = qeye(2)

a = tensor(Iq, destroy(N))
sx = tensor(sigmax(), Ic)
sy = tensor(sigmay(), Ic)

P = (1j*(a.dag()-a))/2
X = (a.dag()+a)/2

def U_ideal_operator(u_k):
    Uk = (1j*u_k*P*sx).expm()
    return Uk

def V_ideal_operator(v_k):
    Vk = (1j*v_k*X*sy).expm()
    return Vk

In [50]:
# used to plot wigner later
def plotting_wigner(
    state,
    contour=False,
    fig=None,
    ax=None,
    max_alpha=2,
    cbar=False,
    npts=51,
    vmin = -1,
    vmax = 1
):
    
    xvec = np.linspace(-max_alpha, max_alpha, npts)
    W = wigner(ptrace(state,1) ,xvec , xvec, g = 2 )
    if fig is None:
        fig = plt.figure(figsize=(6, 5))
    if ax is None:
        ax = fig.subplots()
    if contour:
        levels = np.linspace(-1.1, 1.1, 102)
        im = ax.contourf(
            xvec, xvec, W, cmap="seismic", vmin=vmin, vmax=vmax, levels=levels,
        )
    else:
        im = ax.pcolormesh(
            xvec, xvec, W, cmap="seismic", vmin=vmin, vmax=vmax
        )
    
    ax.set_xlabel(r"Re$(\alpha)$")
    ax.set_ylabel(r"Im$(\alpha)$")
    ax.grid()
    # ax.set_title(title)

    fig.tight_layout()
    if cbar:
        fig.subplots_adjust(right=0.8, hspace=0.25, wspace=0.25)
        # todo: ensure colorbar even with plot...
        # todo: fix this colorbar

        cbar_ax = fig.add_axes([0.85, 0.225, 0.025, 0.65])
        ticks = np.linspace(-1, 1, 5)
        fig.colorbar(im, cax=cbar_ax, ticks=ticks)
        cbar_ax.set_title(r"$\frac{\pi}{2} W(\alpha)$", pad=10)
    ax.set_aspect("equal", adjustable="box")

In [ ]:
# define cost function that is minimized and run optimize.minimize with a random intial guesses.
# run the optimization many times and print the found uvs and the achieved squeezing in each quadrature.
for i in range(3):
    def varianceP(uvs):
        u_1 = uvs[0]
        v_1 = uvs[1]
        u_2 = uvs[2]
        v_2 = uvs[3]
        u_3 = uvs[4]
        v_3 = uvs[5]

        psi1 = V_ideal_operator(v_1)*U_ideal_operator(u_1)*tensor(fock(2,0),fock(N,0))
        psi2 = V_ideal_operator(v_2)*U_ideal_operator(u_2)*psi1
        psi3 = V_ideal_operator(v_3)*U_ideal_operator(u_3)*psi2
        #print(expect(P**2,psi3))
        return expect(P**2,psi3)

    # start with random values between 1 and -1
    initial_guess = [random.randrange(-10, 10, 1)/10 for i in range(6)]

    # i tried different methods, SLSQP seems to give the best results
    result = optimize.minimize(varianceP, initial_guess, method= 'SLSQP' )
    if result.success:
        fitted_params = result.x
        print(fitted_params)
    else:
        raise ValueError(result.message)
    
    
    uvs = fitted_params
    def squeezingStateTwoStep(uvs):
        u_1 = uvs[0]
        v_1 = uvs[1]
        u_2 = uvs[2]
        v_2 = uvs[3]
        u_3 = uvs[4]
        v_3 = uvs[5]

        psi1 = V_ideal_operator(v_1)*U_ideal_operator(u_1)*tensor(fock(2,0),fock(N,0))
        psi2 = V_ideal_operator(v_2)*U_ideal_operator(u_2)*psi1
        psi3 = V_ideal_operator(v_3)*U_ideal_operator(u_3)*psi2
    
        return psi3

    state = squeezingStateTwoStep(uvs)
    dbp = -10*np.log10(expect(P**2,state)*4)
    dbx = -10*np.log10(expect(X**2,state)*4)
    print(f"Squeezed in P by {np.round(dbp,3)} dB")#
    print(f"Squeezed in X by {np.round(dbx,3)} dB")#

In [ ]:
## The best uvs we found for 3 steps of UV -> 7.1dB squeezing.
best_uvs = [-0.84571552,  0.61335235,  2.63648988,  0.30560449, -0.91694799, -0.80353261]
state = squeezingStateTwoStep(best_uvs)
plotting_wigner(state)
dbp = -10*np.log10(expect(P**2,state)*4)
dbx = -10*np.log10(expect(X**2,state)*4)
print(f"Squeezed in P by {np.round(dbp,3)} dB")#
print(f"Squeezed in X by {np.round(dbx,3)} dB")#

### Question: What are the best parameters you found and how did you find them?